# Creating command line tools CLI

## The `click` module

For System Administrators, command line tools are their daily bread. This section helps to create beautiful command line interfaces in a composable way with as little code as necessary. The [click module](https://click.palletsprojects.com/en/7.x/) is the most popular among the many that exist, e.g. [argparse](https://click.palletsprojects.com/en/7.x/why/#why-not-argparse), [docopt](https://click.palletsprojects.com/en/7.x/why/#why-not-docopt-etc) among others.

The `click` module not only makes it relatively easy to create nice and **extendible command line interfaces**, but also implements many best practices. For example, it automagically creates a **help page** for every command and subcommand, handles flags and passwords in a consistent and predictable way and generates human-friendly errors.

But before we start, we need to understand how command line utilities usually work. They consists of three parts:

* `command`
* `option(s)`
* `argument`

The `command` is the **verb**, it explains what we would like to do, e.g. `import-datasets` or `copy-logfiles`. You can offer **sub-commands** if you need to. You can completely omit commands, if your utility is doing only one thing.

The `options` (one, many or none) define more precisely how the utility or the chosen command should behave. Options start with either:
* one dash followed by a letter, e.g. `-a`, `-d`
* two dashes followed by a word,  e.g. `--add`, `--delete`
* options containing two words: `--show-keys`
* Typically most options have a short and a long version
* ... so users can either write `-a` or `--add`.
* Options can be followed by an argument or can act as a switch, like the `--verbose` switch

The `argument` is the **subject**, often a filename, but it can be a simple string too. The `argument` follows after each `option`, but can also follow after a command.

### Installation of `click`

The installation of the module is easy:

```bash
pip install -U click
```

Note that
* `install` is the command
* `-U` is an option, a shorthand for `--upgrade` which forces pip to install the latest version of that module
* `click` is the argument, in our case the name of the module

Documentation of the `click` module can be found [here](https://click.palletsprojects.com/en/7.x/).

## Your first command-line tool

Your first module does a lot of useful things:

* it prompts for your name
* it greets you the many times you want
* it offers a polite version

Open the file [hello_click.py](hello_click.py) or look at the code below.

```python
import click

@click.command()
@click.option("--count",  "-c", default=1, help="Number of greetings.")
@click.option("--polite", "-p", is_flag=True)
@click.option("--name",   "-n", help="The person to greet.")
def hello(count, polite, name):
    if polite:
        greeting = "Your Serene Highness"
    else:
        greeting = "Hello"

    for x in range(count):
        click.echo(f"{greeting} {name}!")

if __name__ == "__main__":
    hello()
```

The first line we already understand:

```python
import click
```

This just imports the click module, ready to be used.

Next, we discover some new elements right above the function:

```python
@click.command()
@click.option("--count",  "-c", default=1, help="Number of greetings.")
@click.option("--polite", "-p", is_flag=True)
@click.option("--name",   "-n", help="The person to greet.")
def hello(count, polite, name):
```

These elements are so called **decorators**. Their functionality is to **wrap** a function or a method. Before the `def hello()` function is being executed, first the `click.command()` is executed, then three times a `click.option()` decorator, before we actually see the function definition.

The options `--count`, `--polite` and `--name` are passed as parameters `count`, `polite` and `name` to our  function, in the order they appear.

You can also define kebab-style options, e.g. `--test-only` which then will be passed as a `test_only` parameter to your function.

```python
def hello(count, polite, name):
    if polite:
        greeting = "Your Serene Highness"
    else:
        greeting = "Hello"
        
    if not name:
        name = "unknown stranger"

    for x in range(count):
        click.echo(f"{greeting} {name}!")
```

The rest of the code in this method prints out a message a number of times, using the `click.echo()` method and the convenient [f-string notation](https://realpython.com/python-f-strings/)

### The entry point of your script: `__name__ == '__main__'`

At the end of our small script, you see one of the more confusing things in Python is this pattern:

```python
if __name__ == "__main__":
    hello()
```

It's called the **entry point** of a script. It tells the interpreter to call the `hello()` function.

### Exercise 1

* Try to remove the entry point from your [hello_click.py](hello_click.py) script and then run it with Python interpreter `python hello_click.py`.
* Observe: what happens?
* try to introduce an obvious error (e.g. `result = 100/0`, either inside or outside the function and run it again.
* Can you explain the behaviour?

**Explanation**

The Python interpreter first performs a syntax check on your script. After the syntax check it imports the click module, it wraps our `hello()` function with 4 decorators, makes the function `hello` available to the program. Finally it reaches the end of the script and exits.

If any error occurs in the **main section** of the program, the Python interpreter halts and prints out a stacktrace. Non syntax-errors *inside* a function or method are *not* detected, they only occur when a function or method is *executed*.

After the compilation step, we have to tell the Python interpreter what to do next, e.g. execute our function. For this we could also simply add:

```python
hello()
```

However, that `hello()` function call then would **always be executed**, even when we only want to import the script from another script. So we need a way to decide, in which **context** our script was called. That's what the

```python
if __name__ == "__main__":
    hello()
```

statement is for. The special variable `__name__` contains the value `__main__` if the script was the first thing that got started. On the other hand, if our script got `import`ed by another script or module, the `__name__` variable contains the name of the importer. In this case, we probably don't want to run anything automatically.

### Boolean values (flags)

Often we want to adjust the behaviour of our command line tool by providing boolean flags. The presence of the flag will set it to True:

```python

@click.option(
    '-d', '--delete',
    is_flag=True,
    help='delete this user'
)
```



### Passwords

When you enter a password or some other confidential value, you don't want to see on the screen what you write. The `hide_input=True` parameter will prevent this.

When resetting passwords, one often needs to enter the new password twice, in case you entered a typo. Use `confirmation_prompt=True` to show the password prompt a second time and compare the two inputs.

```python
@click.option(
    '--password', 
    prompt=True,                 # does what it says: it prompts
    confirmation_prompt=True,    # the password has to be entered twice
    hide_input=True              # do not show the input characters
)
```


## Arguments

[Arguments](https://click.palletsprojects.com/en/8.0.x/arguments/) work almost like options with the difference that they are *positional*. By adding the parameter `nargs=-1`, you can add as many values for one argument (only works for *one* argument, as it eats up all other arguments):

Example:

```python
@click.command()
@click.argument('src', nargs=-1)
@click.argument('dst', nargs=1)
def copy(src, dst):
    """Move file SRC to DST."""
    for fn in src:
        click.echo(f"move {fn} to folder {dst}")
```

### Exercise 2

modify hello_click.py

- add a help text for the `--polite` option and test it
- add a `prompt` parameter to obtain a name, if it is not provided
- add an *optional* [file argument](https://click.palletsprojects.com/en/7.x/arguments/#file-path-arguments) (with `nargs=-1`)
- if the argument is present, open the content of the file and print it after the greeting

## Sub-Commands

Sub-commands are a way to conveniently group options for a given task. For example, the [ethz-iam-webservice](https://gitlab.ethz.ch/vermeul/ethz-iam-webservice) CLI offers the `person`, `user` and the `group` command. Below you see some examples how the utility takes a **command** followed by an **argument** and some **options**:

```bash
$ iam person swen@ethz.ch
$ iam user vermeul
$ iam user vermeul --grant-service LDAPS
$ iam group my_group
$ iam group my_group -a new_user1 -a new_user2 -r old_user3
$ iam group my_group --delete
```

### How to implement a sub-command

1. define the main entry function, e.g. `def do_something()`
2. decorate it with the `@click.group()` decorator
3. define the subcommand function, e.g. `def sing()`
4. decorate it with the `@do_something.command()` decorator

### How to pass context

From the [documentation](https://click.palletsprojects.com/en/8.0.x/commands/#nested-handling-and-contexts)

```python
@click.group()
@click.option('--debug/--no-debug', default=False)
@click.pass_context
def cli(ctx, debug):
    # ensure that ctx.obj exists and is a dict (in case `cli()` is called
    # by means other than the `if` block below)
    ctx.ensure_object(dict)

    ctx.obj['DEBUG'] = debug

@cli.command()
@click.pass_context
def sync(ctx):
    click.echo(f"Debug is {'on' if ctx.obj['DEBUG'] else 'off'}")

if __name__ == '__main__':
    cli(obj={}`
```

### Exercise 3

Open [click_commands.py](click_commands.py)

- observe how the two sub-commands are defined
- try to pass some options and arguments to the sub-commands
- implement an example which takes a value of the main option and passes it to the sub-command

# register a command line utility

When you install the [ethz-iam-webservice]((https://gitlab.ethz.ch/vermeul/ethz-iam-webservice/-/tree/master)) utility using 

```
pip install ethz-iam-webservice
```

you will end up with a command line tool named `iam`. How did that happen?

### Package configuration, the traditional way

When you run a `pip install`, the included `setup.py` script is executed. It contains a lot of options, among them the [entry_points](https://setuptools.pypa.io/en/latest/userguide/entry_point.html) with the `console_scripts`:

```python
    entry_points={
        'console_scripts' : [
            'iam=ethz_iam_webservice.main:cli'
        ]
    }
```

The setuptools will create a small python script (a wrapper script) for you and store it in the `bin/` directory next to your Python interpreter. That Python script has the name `iam` which executes `ethz_iam_webservice.main:cli`:

```
ethz_iam_webservice.main:cli
      ^              ^    ^
      |              |    |
  module name   main.py  method name
```

### Package configuration, the new way

Since pip version v.21.1, the **build** and the **setup** of a package are separated.

Instead of having one single `setup.py`, we have

* `pyproject.toml`
   * declares which build system should be used
   * contains pylint, tox, formatting etc. instructions
   * [documentation](https://pip.pypa.io/en/stable/reference/build-system/pyproject-toml/)
* `setup.cfg` the package information, which contains metadata, contents, dependencies, etc.

Both files are written in a INI-style format, with a section title in `[brackets]` and key-value pairs:

`pyproject.toml`
```ini
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"
```

`setup.cfg`
```ini
[metadata]
name = my_doings
version = 0.0.1

[options]
packages = my_doings

[options.entry_points]
console_scripts =
    do_something = my_doings.click_commands:do_something
```

### Create the package (done ;-)

1. create a folder `my_doings` and move (or copy) `click_commands.py` into that folder
2. inside `my_doings`, create an empty file `__init__.py`
3. create a configuration file `setup.cfg` as shown above, add the entry point:

```ini
[options.entry_points]
console_scripts =
    do_something = my_doings.click_commands:do_something
```

4. create a project file `pyproject.toml` as shown above

### Exercise 4: install your command-line tool locally

**6a.** on a machine *without* privileges:
   - use `pip install . --prefix local_python_packages` to specify a folder where packages are installed
      - Notice how a `local_python_packages` folder is created
   - add the `local_python` folder to your `PYTHONPATH` (make sure you pick the right Python version!)
      - `export PYTHONPATH="$PWD/local_python_packages/lib/python3.9/site-packages"`
   - add this path to your `PATH`:
      - `export PATH="$PWD/local_python_packages/bin:$PATH"`
   - try out the new utility:
      - `do_something`

**6b.** on a machine with install privileges (e.g. your own machine):
   -  install the utility with from the command line with `pip install .`
   - try it out! The new command `do_something` should now be available inside the console

# execute shell commands from Python

While `click` module allows us to create complex command line programs, we still often have to rely on existing tools, i.e. **we need to execute a command on the shell** from within Python. To achieve this, we use the [subprocess module](https://docs.python.org/3/library/subprocess.html), a module from the standard library which faced a lot of changes in the past and is sometimes not very straight forward to use.

## Execute a command and capture the output


the following **only works with Python 3.7 and onwards:** 

```python
import subprocess
result = subprocess.run(["ls", "-la"], capture_output=True)
print(result.stdout.decode("utf-8"))   # stdout is in bytes, we need to decode
```

For **up to Python 3.6** use this:

```python
import subprocess
result = subprocess.run(["ls", "-la"], stdout=subprocess.PIPE)
print(result.stdout.decode("utf-8"))   # stdout is in bytes, we need to decode
```


## Pipe a command to another 

In Unix, piping the results of one command into another is one of the biggest inventions since the invention of the surfboard. Unfortunately, in Python, piping is not very straight-forward to get it right:

1. start a subprocess p1 for the first command
2. retrieve the `stdout` from p1
3. start a subprocess p2, feed the `p1.stdout` into the `stdin` of p2
4. close the `stdout` of p1
5. invoke the `p2.communicate()` method
6. retrieve the output of p2

Here is a simple example which actually works, so you don't have to google it yourself.

```python
import subprocess

p1 = subprocess.Popen(["ls", "-la"], stdout=subprocess.PIPE)
p2 = subprocess.Popen(
    ["grep", "hello"], 
    stdin=p1.stdout, 
    stdout=subprocess.PIPE
)
p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
output = p2.communicate()[0]
print(output.decode('utf-8'))
```
